<a href="https://colab.research.google.com/github/beccarebecca/GAN-learning-assignment/blob/master/ga_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

!pip install tensorflow==2.0.0-alpha0
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os 
from IPython import display
import PIL
import tensorflow as tf
from tensorflow.keras import layers
import time
from __future__ import absolute_import,division,print_function, unicode_literals
from keras.optimizers import Adam

from keras import initializers
!pip install -q imageio
from keras.layers import LeakyReLU

print(tf.__version__)


In [0]:

bufferS = 60000
batchS = 256

from keras.datasets import fashion_mnist
(train_img, train_labes), (_, _) = tf.keras.datasets.fashion_mnist.load_data()

In [0]:
train_img = train_img.reshape(train_img.shape[0], 28, 28, 1).astype('float32')
train_img = (train_img - 127.5)/ 127.5

In [0]:

# shuffle the data
Shuffle_train_set = tf.data.Dataset.from_tensor_slices(train_img).batch(bufferS)

init = initializers.RandomNormal(stddev = 0.02)


In [0]:
# making a generator
# up sampling noise to make the right size from random pixels.

def generatorModelMaker():
  mod = tf.keras.Sequential()
  
  mod.add(layers.Dense(7*7*256, use_bias = False, input_shape = (100,)))
  mod.add(layers.BatchNormalization())
  
  mod.add(layers.LeakyReLU(alpha=0.3))
  mod.add(layers.Reshape((7,7,256)))
  assert mod.output_shape ==(None,7, 7, 256)
  
  mod.add(layers.Conv2DTranspose(128, (5, 5,) , strides = (1, 1), padding = 'same', use_bias =  False))
  assert mod.output_shape ==(None,7, 7, 128)
  mod.add(layers.BatchNormalization())
  

  
  mod.add(layers.LeakyReLU(alpha=0.3)) 
  
  mod.add(layers.Conv2DTranspose(64, (5, 5,) , strides = (2, 2), padding = 'same', use_bias =  False))
  assert mod.output_shape ==(None,14, 14, 64)
  mod.add(layers.BatchNormalization())
  
  
  
  mod.add(layers.LeakyReLU(alpha=0.3))
  
  mod.add(layers.Conv2DTranspose(1,(5,5), strides = (2,2), padding = 'same' , use_bias = False, activation='tanh'))
  assert mod.output_shape ==(None,28, 28, 1)
  
  return mod
    
    

In [0]:
generate = generatorModelMaker()
z = np.random.normal(loc= 0 , scale = 1, size = (batchS, 100))

GenImg = generate(z, training = False)
#plt.imshow(GenImg[0,:,:, 0], cmap = 'gray')



In [0]:
def detective_model():
  detective = tf.keras.Sequential()
  detective.add(layers.Conv2D(64, (5,5), strides = (2, 2), padding = 'same', input_shape=[28,28,1]))
  detective.add(layers.LeakyReLU(alpha=0.3))
  detective.add(layers.Dropout(0.3))
  
  detective.add(layers.Conv2D(128, (5,5), strides = (2, 2), padding = 'same'))
  detective.add(layers.LeakyReLU(alpha=0.3))
  detective.add(layers.Dropout(0.3))
  detective.add(layers.Flatten())
  detective.add(layers.Dense(1))
  
  return detective
  
    

In [0]:
verdict = detective_model()
opinion = verdict(GenImg)
print(opinion)

In [0]:
crossEntropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)
                                                 

In [0]:
def DiscrLoss(rOutput, wOutput):
  correctLoss = crossEntropy(tf.ones_like(rOutput),rOutput)
  sneakyLoss = crossEntropy(tf.zeros_like(wOutput),wOutput)
  total = correctLoss + sneakyLoss
  return total

In [0]:
def genLoss(wOutput):
  return crossEntropy(tf.ones_like(wOutput),wOutput)



genOptimizer = tf.keras.optimizers.Adam(1e-5)
detectiveOptimizer = tf.keras.optimizers.Adam(1e-5)

In [0]:
checkDir = "./checks"
checkMerge = os.path.join(checkDir, "pt")
pointS = tf.train.Checkpoint(genOptimizer = genOptimizer,detectiveOptimizer = detectiveOptimizer, generate = generate,verdict = verdict )

In [0]:
EPOCHS = 50
genDim = 100
numberOfGenExamples = 16
seed = tf.random.normal([numberOfGenExamples, genDim])

In [0]:
@tf.function
def stepS(imgs):
  noise = tf.random.normal([batchS, genDim])
  with tf.GradientTape() as genTape, tf.GradientTape() as detTape:
    genImgs = generate(noise, training = True)
    correctOutput = verdict(imgs, training= True)
    sneakyOutput = verdict(genImgs, training = True)
    GenLoss = genLoss(sneakyOutput)
    DetLoss = DiscrLoss(correctOutput,sneakyOutput)
  derivitivesGen = genTape.gradient(GenLoss, generate.trainable_variables)
  derivitivesdet = detTape.gradient(DetLoss, verdict.trainable_variables)
  genOptimizer.apply_gradients(zip( derivitivesGen, generate.trainable_variables))
  detectiveOptimizer.apply_gradients(zip( derivitivesdet, verdict.trainable_variables))

In [0]:
def train(data, epoch):
  for ep in range(epoch):
    start = time.time()
    for imgB in data:
      stepS(imgB)
    
    display.clear_output(wait = True)
    generateImages(generate, ep + 1, seed)
    if (ep + 1) % 20 ==0:
      pointS.save(file_prefix = checkMerge)
      print('epoch {} id {} sec'. format( ep + 1,time.time()-start))
    display.clear_output(wait= True)
    generateImages(generate, ep, seed)

In [0]:
def generateImages(mod, epoch, t_input):
  evaluator = mod(t_input, training = False)
  pic = plt.figure(figsize=(4,4))
  for i in range (evaluator.shape[0]):
    plt.subplot(4,4, i + 1)
    plt.imshow(evaluator[i, :,:,0] * 127.5 + 127.5, cmap = 'gray')
  plt.savefig("image{:04d}.png".format(epoch))
    

In [0]:
train(Shuffle_train_set, EPOCHS)

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkDir))

In [0]:
def Final_img(num):
  return PIL.image.open("image{:04d}.png". num)
Final_img(EPOCH)
Final_img(EPOCH - 1)